### Tomelilla kommun
* [Issue #38](https://github.com/salgo60/Dogpark_Sweden/issues/38)
* Denna [Notebook Dog38_Tomelilla.ipynb](https://github.com/salgo60/Dogpark_Sweden/blob/main/notebook/Dog38_Tomelilla.ipynb)
* karta [Tomelilla_dog_38.html](https://raw.githack.com/salgo60/Dogpark_Sweden/main/notebook/Tomelilla_dog_38.html)

In [6]:
import requests
import json
import binascii
import folium
import geopandas as gpd
import pandas as pd
from shapely import wkb

# 1. Hämta JSON-data från Tomelilla
url = "https://data.tomelilla.se/rowstore/dataset/1275d7eb-efdf-4892-a864-4f3a40d72594/json"
data = requests.get(url).json()["results"]

# 2. Skapa DataFrame
df = pd.DataFrame(data)

# 3. Konvertera WKB-hex till Shapely-geometri
df["geometry"] = df["geom"].apply(lambda x: wkb.loads(binascii.unhexlify(x)) if x else None)

# 4. Gör till GeoDataFrame i SWEREF 99 TM (EPSG:3006)
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:3006")

# 5. Konvertera till WGS84 (EPSG:4326) för Folium
gdf = gdf.to_crs(epsg=4326)

# 🧩 Extrahera punkter ur MultiPoint (tar första punkten)
gdf["point"] = gdf["geometry"].apply(
    lambda g: list(g.geoms)[0] if g.geom_type == "MultiPoint" else g
)

# 6. Skapa Folium-karta centrerad på Tomelilla
center = [gdf["point"].y.mean(), gdf["point"].x.mean()]
m = folium.Map(location=center, zoom_start=13, tiles="OpenStreetMap")

# 7. Lägg till punkter
for _, row in gdf.iterrows():
    p = row["point"]
    popup_text = (
        f"<b>Hundlatrinpåse</b><br>"
        f"<b>ID:</b> {row['﻿skotselobj_id']}<br>"
        f"<b>Utförare:</b> {row['utforare'] or '-'}<br>"
        f"<b>Datum:</b> {row['inskrivet']}"
    )
    folium.Marker(
        location=[p.y, p.x],
        popup=popup_text,
        icon=folium.Icon(color="green", icon="paw", prefix="fa")
    ).add_to(m)

# 8. Spara till HTML
m.save("Tomelilla_dog_38.html")

print("✅ Kartan Tomelilla_dog_38.html har skapats!")


✅ Kartan Tomelilla_dog_38.html har skapats!


In [7]:
m